In [ ]:
import utils
import numpy as np
import cv2
import open3d as o3d
import time
import os
import pandas as pd

In [ ]:
# Input No. 1 Model for Disparity Estimation - Load ONCE
MODEL_NAME = 'RAFT-Stereo'
MODEL_PATH = "RAFT-Stereo/models/raftstereo-realtime.pth"
model = utils.load_model(MODEL_NAME, MODEL_PATH)

In [ ]:
# Input No. 2 Stereo Camera Parameters - Load ONCE
stereo_params = np.load('stereo_params.npz')
IMG_SIZE = (894, 714)
# Calculate Undistortion and Rectification Remapping
K_L = stereo_params['K_L']
D_L = stereo_params['D_L']
K_R = stereo_params['K_R']
D_R = stereo_params['D_R']
R_L = stereo_params['R_L']
R_R = stereo_params['R_R']
P_L = stereo_params['P_L']
P_R = stereo_params['P_R']
Q = stereo_params['Q']

mapL1, mapL2 = cv2.initUndistortRectifyMap(K_L, D_L, R_L, P_L, IMG_SIZE, cv2.CV_16SC2)
mapR1, mapR2 = cv2.initUndistortRectifyMap(K_R, D_R, R_R, P_R, IMG_SIZE, cv2.CV_16SC2)

In [ ]:
# Input No. 3 3D CT/Model Files - Load ONCE
STL_FILES = []
STL_PATH = "G:\My Drive\Dataset\stl"
for file in os.listdir(STL_PATH):
    if file.endswith('.vtk') or file.endswith('.stl'):
        STL_FILES.append(file)

In [ ]:
# Input No. 4 Buffly to Endoscope Transformation (from System Calibration) - Load ONCE
BF_to_EN_test = np.load("BF_to_EN_test.npy")

In [ ]:
# Input No. 5 CT to OT Transformation (from CT/MR Registration) - Load ONCE
CT_to_OT = np.array([[-0.0102021,-0.998141,0.0600825,-67.1015],
                        [0.99575,-0.0156413,-0.0907656,-492.36],
                        [0.0915366,0.0589012,0.994058,1882.09],
                        [0,0,0,1]])

# The following should be running and updating in REAL-TIME using loop or so

In [ ]:
# Input No. 6 Stereo Images
# Here we just read from local files for testing
TEST_LEFT_IMG = "G:\My Drive\Dataset\davinci_Xi_05282025\KidneyORXI\calibration_images\cam1_cropped\\frame_15.png"
TEST_RIGHT_IMG = "G:\My Drive\Dataset\davinci_Xi_05282025\KidneyORXI\calibration_images\cam2_cropped\\frame_15.png"

In [ ]:
FRAMENAME = os.path.basename(TEST_LEFT_IMG)
OT_TRANS_PATH = "G:\My Drive\Dataset\davinci_Xi_05282025\KidneyORXI\calibration_images" + "\\transforms.csv"
OT_transformations = pd.read_csv(OT_TRANS_PATH)
datapoint = OT_transformations[OT_transformations[' frame name'] == FRAMENAME]
OT_to_BF_raw = datapoint[datapoint['Tool Type'] == 'E']

# Input No. 2 Butterfly Pose (Quaternions)
# Here we just read from local files for testing
OT_to_BF_qt = OT_to_BF_raw[['q0','qx', 'qy', 'qz', 'tx', 'ty', 'tz']]

In [ ]:
# Calculating Disparity
disp = utils.compute_disparity(
            MODEL_NAME, model,
            TEST_LEFT_IMG, TEST_RIGHT_IMG,
            mapL1, mapL2, mapR1, mapR2
        )

In [ ]:
# Generating Point Cloud
pcd = utils.generate_pointcloud(
            MODEL_NAME, disp, TEST_LEFT_IMG, mapL1, mapL2, stereo_params['Q']
        )

In [ ]:
# Transforming CT/MR stl files to Endoscope Frame
pcd, mesh_CT = utils.transform_CTMR(STL_PATH, STL_FILES, OT_to_BF_qt, BF_to_EN_test, pcd, CT_to_OT)

In [ ]:
# Visualize Point Cloud and Mesh
utils.visualize_pcd_mesh(
    pcd, mesh_CT,
    front=(0.0, 0.0, 0.001),
    lookat=(0.0, 0.0, 0.0),
    up=(0.0, 1.0, 0.0)
)